### 3차 다항식을 사용하여 y=sin(x)에 근사하기

In [2]:
# using numpy
import numpy as np
import math 

x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# 가중치 초기화
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # 순전파 단계: 예측값 y를 계산합니다
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # 손실(loss)을 계산하고 출력합니다
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # 손실에 따른 a, b, c, d의 변화도(gradient)를 계산하고 역전파합니다.
    grad_y_pred = 2.0 * (y_pred - y) # loss 함수의 미분
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # 가중치를 갱신합니다.
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

    print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')



Result: y = 0.796616815235709 + -1.301324577548003 x + -2.1390762477480822 x^2 + -0.13980943343079694 x^3
Result: y = 0.8216076263035502 + -1.2692679265248743 x + -1.982543752077429 x^2 + 0.05431000744580328 x^3
Result: y = 0.8444365281340053 + -1.252791024432213 x + -1.8385630184669137 x^2 + 0.13896358388813548 x^3
Result: y = 0.8652775081893139 + -1.243141193608024 x + -1.7061254864007687 x^2 + 0.17568517661318933 x^3
Result: y = 0.8842905725786964 + -1.2364858180915663 x + -1.5843036389274576 x^2 + 0.19141912067558764 x^3
Result: y = 0.9016228695413467 + -1.2311466690342991 x + -1.4722444903428855 x^2 + 0.19796374785653295 x^3
Result: y = 0.9174097226510998 + -1.2263888764661544 x + -1.3691635971757516 x^2 + 0.20048528735518184 x^3
Result: y = 0.9317755809971529 + -1.2218906465975046 x + -1.274339550424706 x^2 + 0.2012459161373165 x^3
Result: y = 0.944834893012296 + -1.2175110626581287 x + -1.1871089103759564 x^2 + 0.20123619040313012 x^3
Result: y = 0.9566929100840115 + -1.21318840

그러나 numpy는 GPU 사용 불가

In [3]:
# using Tensor
import torch
import math


dtype = torch.float
device = torch.device("cuda:5")

# 무작위로 입력과 출력 데이터를 생성합니다
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# 무작위로 가중치를 초기화합니다
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # 순전파 단계: 예측값 y를 계산합니다
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # 손실(loss)을 계산하고 출력합니다
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # 손실에 따른 a, b, c, d의 변화도(gradient)를 계산하고 역전파합니다.
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # 가중치를 갱신합니다.
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')


99 1893.031494140625
199 1317.993408203125
299 919.26416015625
399 642.5203857421875
499 450.26531982421875
599 316.58489990234375
699 223.55206298828125
799 158.7528839111328
899 113.5825424194336
999 82.07058715820312
1099 60.07050323486328
1199 44.70014190673828
1299 33.95420837402344
1399 26.436471939086914
1499 21.173744201660156
1599 17.487438201904297
1699 14.903852462768555
1799 13.092111587524414
1899 11.820980072021484
1999 10.928703308105469
Result: y = 0.04626116529107094 + 0.8429447412490845 x + -0.007980817928910255 x^2 + -0.09136801213026047 x^3


역전파를 직접 구현하지 않고 autograd를 이용해 자동화

In [4]:
# -*- coding: utf-8 -*-
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # GPU에서 실행하려면 이 주석을 제거하세요

# 입력값과 출력값을 갖는 텐서들을 생성합니다.
# requires_grad=False가 기본값으로 설정되어 역전파 단계 중에 이 텐서들에 대한 변화도를
# 계산할 필요가 없음을 나타냅니다.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# 가중치를 갖는 임의의 텐서를 생성합니다. 3차 다항식이므로 4개의 가중치가 필요합니다:
# y = a + b x + c x^2 + d x^3
# requires_grad=True로 설정하여 역전파 단계 중에 이 텐서들에 대한 변화도를 계산할 필요가
# 있음을 나타냅니다.
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # 순전파 단계: 텐서들 간의 연산을 사용하여 예측값 y를 계산합니다.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # 텐서들간의 연산을 사용하여 손실(loss)을 계산하고 출력합니다.
    # 이 때 손실은 (1,) shape을 갖는 텐서입니다.
    # loss.item() 으로 손실이 갖고 있는 스칼라 값을 가져올 수 있습니다.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # autograd 를 사용하여 역전파 단계를 계산합니다. 이는 requires_grad=True를 갖는
    # 모든 텐서들에 대한 손실의 변화도를 계산합니다.
    # 이후 a.grad와 b.grad, c.grad, d.grad는 각각 a, b, c, d에 대한 손실의 변화도를
    # 갖는 텐서가 됩니다.
    loss.backward()

    # 경사하강법(gradient descent)을 사용하여 가중치를 직접 갱신합니다.
    # torch.no_grad()로 감싸는 이유는, 가중치들이 requires_grad=True 지만
    # autograd에서는 이를 추적하지 않을 것이기 때문입니다.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # 가중치 갱신 후에는 변화도를 직접 0으로 만듭니다.
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 4868.0810546875
199 3232.116455078125
299 2147.37890625
399 1428.0045166015625
499 950.8378295898438
599 634.2637939453125
699 424.18798828125
799 284.7508239746094
899 192.17684936523438
999 130.6995849609375
1099 89.86181640625
1199 62.726253509521484
1299 44.689788818359375
1399 32.69737243652344
1499 24.720712661743164
1599 19.41323471069336
1699 15.880359649658203
1799 13.52767276763916
1899 11.96027660369873
1999 10.915544509887695
Result: y = 0.018495075404644012 + 0.8155782222747803 x + -0.0031907076481729746 x^2 + -0.08747535943984985 x^3


autograd는 대규모 신경망에 사용하기에 low-level임 >> nn module을 사용해 구현

In [7]:
import torch
import math

x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# (x, x^2, x^3)를 위한 텐서를 준비합니다.
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# 위 코드에서, x.unsqueeze(-1)은 (2000, 1)의 shape을, p는 (3,)의 shape을 가지므로,
# 이 경우 브로드캐스트(broadcast)가 적용되어 (2000, 3)의 shape을 갖는 텐서를 얻습니다.

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

loss_fn = torch.nn.MSELoss(reduction='sum')

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):
    y_pred = model(xx)
    loss = loss_fn(y_pred, y)
    if t%100 == 99:
        print(t, loss.item())
    
    model.zero_grad() # 가중치 초기화
    loss.backward() # 가중치 변화도 연산

    # 가중치 갱신
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 1158.486083984375
199 771.8536376953125
299 515.3692016601562
399 345.1856994628906
499 232.23948669433594
599 157.2618408203125
699 107.47614288330078
799 74.40924835205078
899 52.440345764160156
999 37.84022903442383
1099 28.134042739868164
1199 21.679231643676758
1299 17.38501739501953
1399 14.527132987976074
1499 12.624330520629883
1599 11.356884002685547
1699 10.51230525970459
1799 9.94921588897705
1899 9.573636054992676
1999 9.322964668273926
Result: y = 0.009764023125171661 + 0.8368023037910461 x + -0.001684456830844283 x^2 + -0.0904942974448204 x^3


optim에서 제공하는 최적화 함수를 사용해 모델 최적화


In [8]:
# -*- coding: utf-8 -*-
import torch
import math

# 입력값과 출력값을 갖는 텐서들을 생성합니다.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 입력 텐서 (x, x^2, x^3)를 준비합니다.
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# nn 패키지를 사용하여 모델과 손실 함수를 정의합니다.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# optim 패키지를 사용하여 모델의 가중치를 갱신할 optimizer를 정의합니다.
# 여기서는 RMSprop을 사용하겠습니다; optim 패키지는 다른 다양한 최적화 알고리즘을 포함하고 있습니다.
# RMSprop 생성자의 첫번째 인자는 어떤 텐서가 갱신되어야 하는지를 알려줍니다.
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
for t in range(2000):
    # 순전파 단계: 모델에 x를 전달하여 예측값 y를 계산합니다.
    y_pred = model(xx)

    # 손실을 계산하고 출력합니다.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # 역전파 단계 전에, optimizer 객체를 사용하여 (모델의 학습 가능한 가중치인) 갱신할
    # 변수들에 대한 모든 변화도(gradient)를 0으로 만듭니다. 이렇게 하는 이유는 기본적으로 
    # .backward()를 호출할 때마다 변화도가 버퍼(buffer)에 (덮어쓰지 않고) 누적되기
    # 때문입니다. 더 자세한 내용은 torch.autograd.backward에 대한 문서를 참조하세요.
    optimizer.zero_grad()

    # 역전파 단계: 모델의 매개변수들에 대한 손실의 변화도를 계산합니다.
    loss.backward()

    # optimizer의 step 함수를 호출하면 매개변수가 갱신됩니다.
    optimizer.step()


linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 57256.54296875
199 32239.171875
299 17000.78515625
399 7839.765625
499 2996.983642578125
599 1023.338134765625
699 523.4148559570312
799 444.8620910644531
899 398.68218994140625
999 339.31744384765625
1099 269.8515930175781
1199 199.43788146972656
1299 136.83445739746094
1399 86.80420684814453
1499 50.584716796875
1599 27.331363677978516
1699 14.881656646728516
1799 10.01917839050293
1899 9.208883285522461
1999 9.026476860046387
Result: y = 5.806408243103078e-09 + 0.8548386096954346 x + -1.161538421001751e-08 x^2 + -0.09400409460067749 x^3


### 사용자 정의 nn.Module
더 복잡한 모델 구현을 위해 nn.Module와 하위클래스(subclass)로 새로운 module을 정의해야 할 때

In [11]:
import torch
import math

class Polynomial3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        return self.a + self.b*x + self.c*x**2 + self.d*c**3

    def string(self):
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'
    


x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 위에서 정의한 클래스로 모델을 생성합니다.
model = Polynomial3()
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)

for t in range(2000):
    y_pred = model(x)
    loss = criterion(y_pred, y)

    if t%100 == 99:
        print(t, loss.item())

    # 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step() # 가중치 업데이터

print(f'Result:{model.string()}')

99 2637.910888671875
199 1485.9091796875
299 1131.39404296875
399 913.001953125
499 760.7463989257812
599 653.1636352539062
699 577.0433349609375
799 523.1773071289062
899 485.0588684082031
999 458.0841369628906
1099 438.995361328125
1199 425.4870910644531
1299 415.9278564453125
1399 409.1632385253906
1499 404.376220703125
1599 400.9886779785156
1699 398.5914611816406
1799 396.89508056640625
1899 395.694580078125
1999 394.8450927734375
Result:y = 0.04799934849143028 + 0.3035065233707428 x + -0.008280673995614052 x^2 + 1.5611342191696167 x^3


### control flow + weight sharing

파이썬의 control flow를 사용해 loop를 구현하고 순전파 단계에서 동일한 parameter를 여러번 재사용(weight sharing)하는 모델 구현

In [14]:
import random
class DynamicNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        y = self.a + self.b * x + self.c *x **2 + self.d * x ** 3

        for exp in range(4, random.randint(4, 6)):
            y = y + self.e *x **exp

        return y

    def string(self):
        """
        Python의 다른 클래스(class)처럼, PyTorch 모듈을 사용해서 사용자 정의 메소드를 정의할 수 있습니다.
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'


# 입력값과 출력값을 갖는 텐서들을 생성합니다.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# 위에서 정의한 클래스로 모델을 생성합니다.
model = DynamicNet()

# 손실 함수와 optimizer를 생성합니다. 이 이상한 모델을 순수한 확률적 경사하강법(SGD; Stochastic Gradient Descent)으로
# 학습하는 것은 어려우므로, 모멘텀(momentum)을 사용합니다.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    # 순전파 단계: 모델에 x를 전달하여 예측값 y를 계산합니다.
    y_pred = model(x)

    # 손실을 계산하고 출력합니다.
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # 변화도를 0으로 만들고, 역전파 단계를 수행하고, 가중치를 갱신합니다.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')    


1999 2126.017578125
3999 951.7078857421875
5999 426.76788330078125
7999 186.69334411621094
9999 90.67747497558594
11999 45.07236862182617
13999 24.863842010498047
15999 15.908665657043457
17999 11.945972442626953
19999 10.135435104370117
21999 9.446438789367676
23999 9.107473373413086
25999 8.948843955993652
27999 8.659462928771973
29999 8.591928482055664
Result: y = 0.0029033992905169725 + 0.8531641364097595 x + -0.001079231034964323 x^2 + -0.09316376596689224 x^3 + 0.00011598577111726627 x^4 ? + 0.00011598577111726627 x^5 ?
